In [1]:
from tensorflow.keras import layers, models, Model, Sequential

2022-03-15 16:36:43.994621: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


![TensorFlow中padding讲解](https://s2.loli.net/2022/03/11/Pw9GLDY1SiWsXrE.png)

In [2]:
"""
Keras Functional API模式定义模型:https://www.jianshu.com/p/90132fdb9a4a(详解)
"""
def AlexNet_v1(im_height=224, im_width=224, num_classes=1000):
    # 1.首先你需要定义一个输入(tensorflow中tensor的通道排序是NHWC):
    input_image = layers.Input(shape=(im_height, im_width, 3), dtype='float32') # output[None, 224, 224, 3]
    
    # 2.其次，定义输出：
    '''
    1）layers.ZeroPadding2D详解：
    ① https://blog.csdn.net/weixin_43533058/article/details/109221535
    ② https://www.cnblogs.com/LGJC1314/p/13403811.html
    2）为什么要进行手动为图像进行padding处理，而不采用TensorFlow默认的padding方式(即'valid'or'same')？
    ①原因见上图；
    ②手动padding之后，图像尺寸由[224, 224]变为了[227, 227]，
    此时运用padding='VALID'方式，可算出(227-11+1)/4=54.5→向上取整，成功得到55的尺寸。
    '''
    x = layers.ZeroPadding2D(((1, 2), (1, 2)))(input_image) # output[None, 227, 227, 3]
    
    x = layers.Conv2D(48, kernel_size=11, strides=4, activation='relu')(x) # output[None, 55, 55, 48]
    x = layers.MaxPool2D(pool_size=3, strides=2)(x) # output[None, 27, 27, 48]
    
    x = layers.Conv2D(128, kernel_size=3, strides=1, padding='same',activation='relu')(x) # output[None, 27, 27, 128]
    x = layers.MaxPool2D(pool_size=3, strides=2)(x) # output[None, 13, 13, 48]

    x = layers.Conv2D(192, kernel_size=3, strides=1, padding='same',activation='relu')(x) # output[None, 13, 13, 192]
    x = layers.Conv2D(192, kernel_size=3, strides=1, padding='same',activation='relu')(x) # output[None, 13, 13, 192]
    x = layers.Conv2D(128, kernel_size=3, strides=1, padding='same',activation='relu')(x) # output[None, 13, 13, 128]
    x = layers.MaxPool2D(pool_size=3, strides=2)(x) # output[None, 6, 6, 128]
    
    x = layers.Flatten()(x) # output[None, 6*6*128]
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2048, activation='relu')(x) # output[None, 2048]
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(2048, activation='relu')(x) # output[None, 2048]
    x = layers.Dense(num_classes)(x) # output[None, num_class]
    prediction = layers.Softmax()(x)
    
    # 3.创建模型
    model = models.Model(inputs=input_image, outputs=prediction)
    return model

In [3]:
"""
贯序模型
"""
class AlexNet_v2(Model):
    def __init__(self, num_classes=1000):
        super(AlexNet_v2, self).__init__()
        self.features = Sequential([
            layers.ZeroPadding2D(((1, 2), (1, 2))), # output(None, 227, 227, 3)
            
            layers.Conv2D(48, kernel_size=11, strides=4, activation="relu"), # output(None, 55, 55, 48)
            layers.MaxPool2D(pool_size=3, strides=2), # output(None, 27, 27, 48)
            layers.Conv2D(128, kernel_size=5, padding="same", activation="relu"), # output(None, 27, 27, 128)
            layers.MaxPool2D(pool_size=3, strides=2), # output(None, 13, 13, 128)
            layers.Conv2D(192, kernel_size=3, padding="same", activation="relu"), # output(None, 13, 13, 192)
            layers.Conv2D(192, kernel_size=3, padding="same", activation="relu"), # output(None, 13, 13, 192)
            layers.Conv2D(128, kernel_size=3, padding="same", activation="relu"), # output(None, 13, 13, 128)
            layers.MaxPool2D(pool_size=3, strides=2) # output(None, 6, 6, 128)
        ])

        self.classifer = Sequential([
            layers.Dropout(0.2),
            layers.Dense(1024, activation="relu"), # output(None, 2048)
            layers.Dropout(0.2),
            layers.Dense(128, activation="relu"), # output(None, 2048)
            layers.Dense(num_classes), # output(None, 5)
            layers.Softmax()
        ])
    
    def call(self, inputs, **kwargs):
        x = self.features(inputs)
        x = layers.Flatten()(x) # output[None, 6*6*128]
        x = self.classifer(x)
        return x